In [1]:
#get data
import numpy as np
from sklearn import manifold
from msmbuilder.example_datasets import FsPeptide
fs_peptide = FsPeptide()
fs_peptide.cache()

In [4]:
import tempfile
import os
import time
os.chdir(tempfile.mkdtemp())

In [3]:
from msmbuilder.dataset import dataset
xyz = dataset(fs_peptide.data_dir + "/*.xtc",
              topology=fs_peptide.data_dir + '/fs-peptide.pdb',
              stride=10)
print("{} trajectories".format(len(xyz)))
# msmbuilder does not keep track of units! You must keep track of your
# data's timestep
to_ns = 0.5
print("with length {} ns".format(set(len(x)*to_ns for x in xyz)))

C:\Users\MinCheol\Anaconda2\lib\site-packages\mdtraj\formats\pdb\pdbfile.py:196: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


28 trajectories
with length set([500.0]) ns


In [9]:
# combine all trajectories into a trajectory "bag"
import mdtraj as md
start = time.time()
frames_bag = []
for idx, trajectories in enumerate(xyz):
    if idx == 0:
        frames_bag = trajectories
    if idx != 0:
        frames_bag = frames_bag.join(trajectories)
temp = xyz[0]
reference_frame = temp.slice(0, copy=True)
frames_bag.superpose(reference_frame)
# Format data matrix
num_frames, num_atoms, num_axis = frames_bag.xyz.shape
X1 = np.reshape(frames_bag.xyz, (num_frames, num_atoms*num_axis))
print(time.time() - start)

17.3070001602


In [24]:
start = time.time();
temp = xyz[0]
_, num_atoms, num_axis = temp.xyz.shape
reference_frame = temp.slice(0, copy=True)
num_features = num_atoms*num_axis;
pre_X = [np.reshape(traj.xyz, (traj.superpose(reference_frame).xyz.shape[0],num_features)) for traj in xyz]
X2 = np.concatenate(pre_X)
print(time.time() - start)

16.3029999733


In [26]:
num_axis

3L

In [19]:
#a pply dimensionality reduction
n_neighbors = 40
n_components = 30
X_iso = manifold.Isomap(n_neighbors, n_components).fit_transform(X)

In [14]:
# use K means to cluster and save data
num_clusters = 97
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=num_clusters).fit(X_iso)

NameError: name 'X_iso' is not defined

In [52]:
# compute XYZ coordinates of cluster centers
cluster_centers = np.empty((num_atoms*num_axis, num_clusters), dtype=float)
for idx in range(num_clusters):
    indices = (kmeans.labels_ == idx)
    cluster_centers[:, idx] = X[indices,:].mean(axis=0)

In [66]:
# save clusters
cluster_centers.dump('C:\\Users\\minch_000\\Documents\\GitHub\\Protein_Dynamics_TJ\\MSM_Builder_Test\\temp.dat')

In [30]:
# reverse engineer clusters
import numpy as np
from sklearn.cluster import KMeans
isomap_clusters = np.load('C:\\Users\\minch_000\\Documents\\GitHub\\Protein_Dynamics_TJ\\MSM_Builder_Test\\isomap_clusters.dat')
kmeans_temp = KMeans(n_clusters=num_clusters).fit(isomap_clusters.transpose())
isomap_clusters = isomap_clusters[:, kmeans_temp.labels_]
assignments = kmeans_temp.predict(X)

In [51]:
# match isomap clusters to msm clusters
pairings = [0 for i in range(97)]
for idx in range(97):
    isomap_center = isomap_clusters[:, idx]
    distances = np.square(np.tile(isomap_center, (100, 1)).transpose() - msm_cluster_centers).sum(axis=0)
    pairings[idx] = np.argmin(distances)

In [71]:
new_assignments = [0 for x in range(len(assignments))]
for idx in range(len(assignments)):
                   new_assignments[idx] = pairings[assignments[idx]]
np.array(new_assignments[0:50])

array([74, 74, 74, 98, 98, 74, 84, 74, 98, 74, 74, 74, 84, 84, 84, 69, 74,
       74, 74, 84, 74, 84, 74, 84, 74, 84, 74, 74, 74, 84, 84, 74, 74, 84,
       84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 74, 84, 84], dtype=int64)

In [72]:
msm_cluster_indices[0:50]

array([73, 73, 73, 23, 73, 23, 73, 23, 73, 73, 73, 73, 73, 73, 73, 73, 73,
       73, 73, 23, 73, 23, 73, 23, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73,
       73, 23, 23, 23, 23, 73, 23, 23, 73, 23, 73, 23, 23, 73, 73, 73])

In [32]:
################################################################################
# Authors: Min Cheol Kim, Christian Choe

# Description: Performs ISOMAP on msm builder tutorial trajectories,
# clusters in reduced dimension, and saves various clustering/dimentionality
# reduction products.

################################################################################

# get data
from msmbuilder.example_datasets import FsPeptide
fs_peptide = FsPeptide()
fs_peptide.cache()

import tempfile
import os
os.chdir(tempfile.mkdtemp())

from msmbuilder.dataset import dataset
xyz = dataset(fs_peptide.data_dir + "/*.xtc",
              topology=fs_peptide.data_dir + '/fs-peptide.pdb',
              stride=10)
print("{} trajectories".format(len(xyz)))
# msmbuilder does not keep track of units! You must keep track of your
# data's timestep
to_ns = 0.5
print("with length {} ns".format(set(len(x)*to_ns for x in xyz)))

#featurization
from msmbuilder.featurizer import DihedralFeaturizer
featurizer = DihedralFeaturizer(types=['phi', 'psi'])
diheds = xyz.fit_transform_with(featurizer, 'diheds/', fmt='dir-npy')

#tICA
from msmbuilder.decomposition import tICA
tica_model = tICA(lag_time=2, n_components=4)
# fit and transform can be done in seperate steps:
tica_model = diheds.fit_with(tica_model)
tica_trajs = diheds.transform_with(tica_model, 'ticas/', fmt='dir-npy')

#histogram

import numpy as np
txx = np.concatenate(tica_trajs)

# clustering
from msmbuilder.cluster import MiniBatchKMeans
clusterer = MiniBatchKMeans(n_clusters=100)
clustered_trajs = tica_trajs.fit_transform_with(
    clusterer, 'kmeans/', fmt='dir-npy'
)

print(tica_trajs[0].shape)
print(clustered_trajs[0].shape)

# combine all trajectories into a trajectory "bag"
frames_bag = []
for idx, trajectories in enumerate(xyz):
    if idx == 0:
        frames_bag = trajectories
    if idx != 0:
        frames_bag = frames_bag.join(trajectories)
num_frames, num_atoms, num_axis = frames_bag.xyz.shape

# Concatenate the trajectories in cluster indices
msm_cluster_indices = np.concatenate(clustered_trajs)

# compute XYZ coordinates of cluster centers
num_clusters=100
msm_cluster_centers = np.empty((num_atoms*num_axis, num_clusters), dtype=float)
X = np.reshape(frames_bag.xyz, (num_frames, num_atoms*num_axis))
for idx in range(num_clusters):
    indices = (msm_cluster_indices == idx)
    msm_cluster_centers[:, idx] = X[indices,:].mean(axis=0)

C:\Users\minch_000\Anaconda2\lib\site-packages\mdtraj\formats\pdb\pdbfile.py:196: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


28 trajectories
with length set([500.0]) ns
(1000L, 4L)
(1000L,)
